In [76]:
import pandas as pd
import numpy as np
# This is on my local machine
pica = pd.read_csv('/Users/jennasparks/Desktop/Picaflor/monitoreo_nido_picaflor_2021.csv')

In [77]:
# Pre-modifications
pica.head()
pica.shape

(324, 15)

In [78]:
# Convert the Fecha column to datetime
pica["FECHA"] = pd.to_datetime(pica["FECHA"])

# Group by NUMERO and select the earliest date and first value of ESPECIE for each group
datospornido = pica.groupby("NUMERO").agg({"FECHA": "min", "ESPECIE": "first"}).reset_index()

# Extract the first two letters of the ESPECIE column and create a new column called especie
datospornido["especie"] = datospornido["ESPECIE"].str[:2]

datospornido = datospornido.drop(columns="ESPECIE")

datospornido = datospornido.rename(columns={"FECHA": "ch_primer", "NUMERO": "numero"})

# Add a new column called ch_primer_j that contains the Julian dates of the ch_primer column
datospornido["ch_primer_j"] = datospornido["ch_primer"].apply(lambda x: x.timetuple().tm_yday)

# Define the new order of columns
new_cols = ['numero', 'especie', 'ch_primer', 'ch_primer_j']

# Reorder the columns
datospornido = datospornido.reindex(columns=new_cols)

# Print the datospornido dataframe
print(datospornido)

    numero especie  ch_primer  ch_primer_j
0        1      sf 2021-07-03          184
1        2      sf 2021-07-15          196
2        3      sf 2021-07-15          196
3        4      sf 2021-07-20          201
4        5      sf 2021-07-20          201
5        6      sf 2021-07-22          203
6        7      ss 2021-07-22          203
7        8      sf 2021-07-31          212
8        9      sf 2021-08-03          215
9       10      ss 2021-08-03          215
10      11      sf 2021-08-05          217
11      12      sf 2021-08-05          217
12      13      sf 2021-08-10          222
13      14      ss 2021-08-10          222
14      15      sf 2021-08-10          222
15      16      ss 2021-08-10          222
16      17      ss 2021-08-10          222
17      18      ss 2021-08-19          231
18      19      ss 2021-08-19          231
19      20      ss 2021-08-31          243
20      21      ss 2021-08-24          236
21      22      ss 2021-08-31          243
22      23 

In [75]:
# Group by NUMERO and POLLUELOS and select the earliest date for each group
ch_prim_pol = pica.groupby(["NUMERO", "POLLUELOS"])["FECHA"].min().reset_index()

# Merge the datospornido dataframe with the ch_prim_pol dataframe
datospornido = pd.merge(datospornido, ch_prim_pol, on="numero", how="left")

# Subtract 20 days from the first check date when there were chicks to infer the lay date, unless it doesn't make sense based on egg check
datospornido.loc[datospornido["FECHA_x"] < datospornido["FECHA_y"] - pd.Timedelta(days=20), "fecha_infer"] = datospornido["FECHA_x"]
datospornido.loc[datospornido["FECHA_x"] >= datospornido["FECHA_y"] - pd.Timedelta(days=20), "fecha_infer"] = datospornido["FECHA_y"] - pd.Timedelta(days=20)

# Drop the FECHA_y column
datospornido = datospornido.drop(columns="FECHA_y")

# Rename the FECHA_x column to ch_primer
datospornido = datospornido.rename(columns={"FECHA_x": "ch_primer"})

print(datospornido)

KeyError: 'numero'